### Free space: FTS

FTS matrix elements only with symmetry factors
        
        FTSMEsym.py

On the note of a general Green's function, have to write the matrix elements more general. In previous computation I have used a lot of the symmetries of the free space Green's function.

In [1]:
from sympy import *
import numpy as np
from sympy.printing.numpy import NumPyPrinter

import re

### Definitions and auxiliary functions

In [2]:
xij, yij, zij, b, eta = symbols("xij yij zij b eta", real=True) #probably need FH and VH symbols too

# indices for summation:
## Caution, cannot use lambda, instead use Lambda
## do not use eta as index, it's the viscosity
alpha, beta, gamma, gamma1, gamma2, gamma3, gamma4, kappa, kappa1, kappa2, mu, nu, nu1, nu2, Lambda, eta2, zeta, sigma = symbols(
    'alpha, beta, gamma, gamma1, gamma2, gamma3, gamma4, kappa, kappa1, kappa2, mu, nu, nu1, nu2, Lambda, eta2, zeta, sigma', integer=True)


## for FH and VH checkout the 'Indexed' package to define them as FH[1], FH[2] etc.

rij = Array([xij, yij, zij])

def norm(rij):
    return sqrt(summation(rij[alpha]**2,(alpha,0,2)))

KroneckerDelta = eye(3)

def Laplacian(G):
    return diff(G, rij[0], rij[0]) + diff(G, rij[1], rij[1]) + diff(G, rij[2], rij[2])

def eps(alpha, beta, gamma):
    return LeviCivita(alpha,beta,gamma)

In [3]:
def Curl(G, alpha, beta): ## the function to take the curl of and the two remaining indices 
    result=0
    for i in range(3):
        for j in range(3):
            result += eps(alpha,i,j)*diff(G(rij,j,beta),rij[i]) ## G is symmetric in its indices
    return result

def CurlCurl(G, alpha, beta): ## the function to take the curl of and the two remaining indices 
    result=0
    for i in range(3):
        for j in range(3):
            for k in range(3):
                for l in range(3):
                    result += eps(alpha,i,j)*eps(beta,k,l)*diff(G(rij,j,l), rij[i], rij[k])
    return result

### Matrix elements

In [4]:
def G(rij, alpha, beta):
    return 1/(8*pi*eta)*(KroneckerDelta[alpha,beta]/norm(rij) + rij[alpha]*rij[beta]/norm(rij)**3)

def delG(rij, alpha, beta, gamma):
    return 1/(8*pi*eta)*((-KroneckerDelta[alpha,beta]*rij[gamma] + KroneckerDelta[alpha,gamma]*rij[beta] + KroneckerDelta[beta,gamma]*rij[alpha])/norm(rij)**3
                         - 3*rij[alpha]*rij[beta]*rij[gamma]/norm(rij)**5)

def lapG(rij, alpha, beta):
    return 1/(4*pi*eta)*(KroneckerDelta[alpha,beta]/norm(rij)**3 - 3*rij[alpha]*rij[beta]/norm(rij)**5)

In [5]:
##auxiliary functions
def dellapG(rij, alpha, beta, kappa1):
    return diff(lapG(rij, alpha, beta), rij[kappa1])

def deldelG(rij, zeta, beta, kappa1, nu):
    return diff(delG(rij, zeta, beta, kappa1), rij[nu])

def deldellapG(rij, alpha, beta, gamma1, kappa1):
    return diff(dellapG(rij,alpha,beta, gamma1), rij[kappa1])

def deldeldelG(rij, alpha, beta, gamma1, gamma2, gamma3):
    return diff(deldelG(rij, alpha, beta, gamma1, gamma2), rij[gamma3])

def deldeldeldelG(rij, alpha, beta, gamma1, gamma2, gamma3, gamma4):
    return diff(deldeldelG(rij, alpha, beta, gamma1, gamma2, gamma3), rij[gamma4])

In [6]:
# GLL
def G1s1s(rij, alpha, beta):
    return G(rij, alpha, beta) + b**2/3*lapG(rij, alpha, beta)

def G1s2a(rij, alpha, beta):
    return -0.5*b*Curl(G, beta, alpha)
#summation(summation(eps(beta, nu, zeta)*delG(rij, alpha, zeta, nu), (nu,0,2)), (zeta,0,2))

def G2a1s(rij, alpha, beta):
    return b*Curl(G,alpha,beta)
#summation(summation(eps(alpha,nu,zeta)*delG(rij,zeta,beta,nu), (nu,0,2)), (zeta,0,2))

def G2a2a(rij, alpha, beta):
    return -0.5*b*b*CurlCurl(G,alpha,beta)
#summation(summation(summation(summation(eps(alpha,mu,kappa)*eps(beta,nu,zeta)*deldelG(rij,kappa,zeta,nu,mu),(mu,0,2)), (kappa,0,2)), (nu,0,2)), (zeta,0,2))

In [7]:
## GL2s and G2sL
def G1s2s(rij, alpha, kappa1, beta):
    return -0.5*b*((delG(rij, alpha, beta, kappa1) + delG(rij, alpha, kappa1, beta)) + 4*b**2/15*(dellapG(rij, alpha, beta, kappa1) + dellapG(rij, alpha, kappa1, beta)))

def G2a2s(rij, alpha, kappa1, beta):
    return -0.5*b*b*(diff(Curl(G, alpha, beta),rij[kappa1]) + diff(Curl(G, alpha, kappa1),rij[beta]))
#summation(summation(eps(alpha,nu,zeta)*(deldelG(rij, zeta, beta, kappa1, nu) + deldelG(rij, zeta, kappa1, beta, nu)), (nu,0,2)), (zeta,0,2))

def G2s1s(rij, alpha, gamma1, beta):
    return 0.5*b*((delG(rij, alpha, beta, gamma1) + delG(rij, gamma1, beta, alpha)) + 4*b**2/15*(dellapG(rij,alpha,beta,gamma1) + dellapG(rij, gamma1, beta, alpha)))

def G2s2a(rij, alpha, gamma1, mu):
    return -0.25*b*b*(diff(Curl(G,mu,gamma1), rij[alpha]) + diff(Curl(G,mu,alpha), rij[gamma1])) #sign change from -> eps(mu, kappa1, beta)
#summation(summation(eps(beta, kappa1, mu)*(deldelG(rij, gamma1, beta, alpha, kappa1) + deldelG(rij, alpha, beta, gamma1, kappa1)), (kappa1, 0, 2)), (beta,0,2))


## G2s2s
def G2s2s(rij, alpha, gamma1, kappa1, beta):
    return -0.25*b*b*((deldelG(rij, alpha, beta, gamma1, kappa1) + deldelG(rij, gamma1, beta, alpha, kappa1)) + 
                      (deldelG(rij,alpha,kappa1, gamma1, beta) + deldelG(rij, gamma1, kappa1, alpha, beta)) +
                      b**2/5*(deldellapG(rij,alpha,beta,gamma1,kappa1) + deldellapG(rij,gamma1,beta,alpha,kappa1)) + 
                      b**2/5*(deldellapG(rij, alpha, kappa1, gamma1, beta) + deldellapG(rij,gamma1,kappa1,alpha,beta)))


def k2s2sSym(rij,alpha,gamma1,kappa1,beta):
    return deldelG(rij,alpha,kappa1,gamma1,beta) + deldelG(rij,alpha,beta,gamma1,kappa1) + deldelG(rij,gamma1,kappa1,alpha,beta) + deldelG(rij,gamma1,beta,alpha,kappa1)

def K2s2s(rij,alpha,gamma1,kappa1,beta):
    return -2*pi*eta*b**3/3*(k2s2sSym(rij,alpha,gamma1,kappa1,beta) + b**2/5*Laplacian(k2s2sSym(rij,alpha,gamma1,kappa1,beta)))

def k1s2sSym(rij, alpha, beta, gamma):
    return delG(rij,alpha,gamma,beta) + delG(rij, alpha, beta, gamma)

def K1s2s(rij, alpha,beta,gamma):
    return -4*pi*eta*b**2/3*(k1s2sSym(rij, alpha, beta, gamma) + Laplacian(k1s2sSym(rij, alpha, beta, gamma)))

def K2a2s(rij, Lambda, beta, gamma):
    return -4*pi*eta*b**3/3*(diff(Curl(G,Lambda,gamma), rij[beta]) + diff(Curl(G,Lambda,beta), rij[gamma])) 



### Systematic method of identifying which symmetry factor has to be applied

In [8]:
def check_duplicates(listOfElems): ## for eg 0,0,1 but will also give true for 0,0,0
    ''' Check if given list contains any duplicates '''
    if len(listOfElems) == len(set(listOfElems)):
        return False
    else:
        return True

def all_equal(iterator): ## for 0,0,0
    return len(set(iterator)) <= 1 



In [9]:
## 3s template
## Number of x,y and z: 18x, 6y and 3z. Want to stop if as quick as possible --> check x, then y, then z
m,n,l = 1,1,2

if check_duplicates([m,n,l]) and not all_equal([m,n,l]): ## two indices are equal
    print('x')
elif not check_duplicates([m,n,l]): ## all indices different
    print('y')
else:                           ## diagonal, all indices equal
    print('z') 

x


In [10]:
## 2s template (same for 3a)
## Number of a and b: 6a and 3b
i,j = 0,1

if not all_equal([i,j]): ## off-diagonal terms
    print('a')
else:                    ## diagonal terms
    print('b')


a


### Matrix assembly

In [11]:
%%time

matG1s1s = Matrix.zeros(3,3)
for i in range(3):
    for j in range(3):
        matG1s1s[i,j] = simplify(G1s1s(rij, i, j))
        
print('G1s1s done')
        
matG2a1s = Matrix.zeros(3,3)
for i in range(3):
    for j in range(3):
        matG2a1s[i,j] = simplify(G2a1s(rij, i, j))
        
print('G2a1s done')
        
matG1s2a = Matrix.zeros(3,3)
for i in range(3):
    for j in range(3):
        matG1s2a[i,j] = simplify(G1s2a(rij, i, j))
        
print('G1s2a done')
        
matG2a2a = Matrix.zeros(3,3)
for i in range(3):
    for j in range(3):
        matG2a2a[i,j] = simplify(G2a2a(rij, i, j))
        
print('G2a2a done')
        
matG1s2s = Matrix.zeros(3,6)
j=0
for i in range(3):
    for k in range(3):
        for l in range(k,3):
            if not all_equal([k,l]):
                matG1s2s[i,j] = 2*simplify(G1s2s(rij, i, k, l))
                
            else:
                matG1s2s[i,j] = simplify(G1s2s(rij, i, k, l))
            j += 1
    j=0
    
print('G1s2s done')

matG2a2s = Matrix.zeros(3,6)
j=0
for i in range(3):
    for k in range(3):
        for l in range(k,3):
            if not all_equal([k,l]):
                matG2a2s[i,j] = 2*simplify(G2a2s(rij, i, k, l))
                
            else:
                matG2a2s[i,j] = simplify(G2a2s(rij, i, k, l))
            j += 1
    j=0
    
print('G2a2s done')
    
matG2s1s = Matrix.zeros(6,3)
i=0
for j in range(3):
    for k in range(3):
        for l in range(k,3):
            if not all_equal([k,l]):
                matG2s1s[i,j] = 2*simplify(G2s1s(rij, k, l, j))
                
            else:
                matG2s1s[i,j] = simplify(G2s1s(rij, k, l, j))
            i += 1
    i=0
    
print('G2s1s done')
    
matG2s2a = Matrix.zeros(6,3)
i=0
for j in range(3):
    for k in range(3):
        for l in range(k,3):
            if not all_equal([k,l]):
                matG2s2a[i,j] = 2*simplify(G2s2a(rij, k, l, j))
                
            else:
                matG2s2a[i,j] = simplify(G2s2a(rij, k, l, j))
            i += 1
    i=0
    
print('G2s2a done')
    
matG2s2s = Matrix.zeros(6,6)
i=0
j=0
for ki in range(3):
    for li in range(ki,3):
        for kj in range(3):
            for lj in range(kj,3):                                          ##could also write as nested if statements -> done at level of (3s)
                if not all_equal([ki,li]) and not all_equal([kj,lj]):
                    matG2s2s[i,j] = 4*simplify(G2s2s(rij, ki, li, kj, lj))
                
                elif not all_equal([ki,li]) and all_equal([kj,lj]):
                    matG2s2s[i,j] = 2*simplify(G2s2s(rij, ki, li, kj, lj))
                    
                elif all_equal([ki,li]) and not all_equal([kj,lj]):
                    matG2s2s[i,j] = 2*simplify(G2s2s(rij, ki, li, kj, lj))
                    
                else:
                    matG2s2s[i,j] = simplify(G2s2s(rij, ki, li, kj, lj))
                j += 1
        j=0
        i += 1 
        
print('G2s2s done')

matK1s2s = Matrix.zeros(3,6)
j=0
for i in range(3):
    for k in range(3):
        for l in range(k,3):
            if not all_equal([k,l]):
                matK1s2s[i,j] = 2*simplify(K1s2s(rij, i, k, l))
                
            else:
                matK1s2s[i,j] = simplify(K1s2s(rij, i, k, l))
            j += 1
    j=0
    
print('K1s2s done')

matK2a2s = Matrix.zeros(3,6)
j=0
for i in range(3):
    for k in range(3):
        for l in range(k,3):
            if not all_equal([k,l]):
                matK2a2s[i,j] = 2*simplify(K2a2s(rij, i, k, l))
                
            else:
                matK2a2s[i,j] = simplify(K2a2s(rij, i, k, l))
            j += 1
    j=0  
    
print('K2a2s done')
    
matK2s2s = Matrix.zeros(6,6)
i=0
j=0
for ki in range(3):
    for li in range(ki,3):
        for kj in range(3):
            for lj in range(kj,3):
                if not all_equal([ki,li]) and not all_equal([kj,lj]):
                    matK2s2s[i,j] = 4*simplify(K2s2s(rij, ki, li, kj, lj))
                
                elif not all_equal([ki,li]) and all_equal([kj,lj]):
                    matK2s2s[i,j] = 2*simplify(K2s2s(rij, ki, li, kj, lj))
                    
                elif all_equal([ki,li]) and not all_equal([kj,lj]):
                    matK2s2s[i,j] = 2*simplify(K2s2s(rij, ki, li, kj, lj))
                    
                else:
                    matK2s2s[i,j] = simplify(K2s2s(rij, ki, li, kj, lj))
                j += 1
        j=0
        i += 1 
        
print('K2s2s done')
        

print('...done')

G1s1s done
G2a1s done
G1s2a done
G2a2a done
G1s2s done
G2a2s done
G2s1s done
G2s2a done
G2s2s done
K1s2s done
K2a2s done
K2s2s done
...done
CPU times: user 1min 22s, sys: 84.5 ms, total: 1min 22s
Wall time: 1min 22s


Create matrices that are zero

Need the following objects

GH1s(force), GH2a(torque), KHH(VH), GHH(FH), 

G1s1s(force), G2a1s(force), G1s2a(torque), G2a2a(torque)
G1sH(FH), K1sH(VH), G2aH(FH), K2aH(FH)

Want to assemble them locally and then print them to a py file. keep placeholders such as force (array of len 3), FH (array of len 20) etc. create these first...

In [12]:
## In python code will be force[i,0], so have to create force as np.zeros([3*Np,1]) object instead of np.zeros([3*Np])
force  = Matrix(MatrixSymbol('force',3,1)) ##create an actual Matrix from it, have to pass to 'Matrix' 
torque = Matrix(MatrixSymbol('torque',3,1))
V2s     = Matrix(MatrixSymbol('V2s',6,1))
F2s     = Matrix(MatrixSymbol('F2s',6,1))

Create block matrices, then multiply with vectors and then print as functions

In [13]:
##diagonal elements
g1s = 1/(6*pi*eta*b)
g2a = 1/(4*pi*eta*b)

g2s = 3/(20*pi*eta*b)

matGo2s2s = diag(g2s,2*g2s,2*g2s,g2s,2*g2s,g2s)

halfMinusk2s = 0.6

matKo2s2s = diag(halfMinusk2s,2*halfMinusk2s,2*halfMinusk2s,halfMinusk2s,2*halfMinusk2s,halfMinusk2s)

Create vectors instead of constructing the full matrix

In [17]:
vecG2s1sF = matG2s1s*force
vecG2s2aT = matG2s2a*torque

vecK2s2sV2s_diag = matKo2s2s*V2s
vecK2s2sV2s = matK2s2s*V2s

vecG2s2sF2s_diag = matGo2s2s*F2s
vecG2s2sF2s = matG2s2s*F2s

vecG1s1sF = matG1s1s*force
vecG2a1sF = matG2a1s*force

vecG1s2aT = matG1s2a*torque
vecG2a2aT = matG2a2a*torque

vecG1s2sF2s = matG1s2s*F2s
vecG2a2sF2s = matG2a2s*F2s

vecK1s2sV2s = matK1s2s*V2s
vecK2a2sV2s = matK2a2s*V2s

_____________

### Write symbolic results to file as reusable python functions

In [18]:
ME_file = "FTSMEsym.py"

with open(ME_file, "w") as text_file:
    print("import numpy", file=text_file)
    print("PI = 3.14159265359\n", file=text_file)
    
    print("def G1s1sF(xij,yij,zij, b,eta, force):\n    return numpy.array({})\n".format(np.reshape(np.asarray(vecG1s1sF),(3,)).tolist()), file=text_file)
    print("def G2a1sF(xij,yij,zij, b,eta, force):\n    return numpy.array({})\n".format(np.reshape(np.asarray(vecG2a1sF),(3,)).tolist()), file=text_file)
    
    print("def G1s2aT(xij,yij,zij, b,eta, torque):\n    return numpy.array({})\n".format(np.reshape(np.asarray(vecG1s2aT),(3,)).tolist()), file=text_file)
    print("def G2a2aT(xij,yij,zij, b,eta, torque):\n    return numpy.array({})\n".format(np.reshape(np.asarray(vecG2a2aT),(3,)).tolist()), file=text_file)
    
    print("def G2s1sF(xij,yij,zij, b,eta, force):\n    return numpy.array({})\n".format(np.reshape(np.asarray(vecG2s1sF),(6,)).tolist()), file=text_file)
    print("def G2s2aT(xij,yij,zij, b,eta, torque):\n    return numpy.array({})\n".format(np.reshape(np.asarray(vecG2s2aT),(6,)).tolist()), file=text_file)
    
    print("def Ko2s2sV2s(b,eta, V2s):\n    return numpy.array({})\n".format(np.reshape(np.asarray(vecK2s2sV2s_diag),(6,)).tolist()), file=text_file)
    print("def K2s2sV2s(xij,yij,zij, b,eta, V2s):\n    return numpy.array({})\n".format(np.reshape(np.asarray(vecK2s2sV2s),(6,)).tolist()), file=text_file)
    
    print("def Go2s2sF2s(b,eta, F2s):\n    return numpy.array({})\n".format(np.reshape(np.asarray(vecG2s2sF2s_diag),(6,)).tolist()), file=text_file)
    print("def G2s2sF2s(xij,yij,zij, b,eta, F2s):\n    return numpy.array({})\n".format(np.reshape(np.asarray(vecG2s2sF2s),(6,)).tolist()), file=text_file)
    
    print("def G1s2sF2s(xij,yij,zij, b,eta, F2s):\n    return numpy.array({})\n".format(np.reshape(np.asarray(vecG1s2sF2s),(3,)).tolist()), file=text_file)
    print("def G2a2sF2s(xij,yij,zij, b,eta, F2s):\n    return numpy.array({})\n".format(np.reshape(np.asarray(vecG2a2sF2s),(3,)).tolist()), file=text_file)
    
    print("def K1s2sV2s(xij,yij,zij, b,eta, V2s):\n    return numpy.array({})\n".format(np.reshape(np.asarray(vecK1s2sV2s),(3,)).tolist()), file=text_file)
    print("def K2a2sV2s(xij,yij,zij, b,eta, V2s):\n    return numpy.array({})\n".format(np.reshape(np.asarray(vecK2a2sV2s),(3,)).tolist()), file=text_file)
    
    
    
## replace things for easier usage with python     
with open(ME_file, 'r') as file:
    filedata = file.read()
    
filedata = filedata.replace('pi', 'PI')
filedata = filedata.replace('sqrt', 'numpy.sqrt')

filedata = re.sub("force\[(\d), 0\]", r"force[\1]", filedata)
filedata = re.sub("torque\[(\d), 0\]", r"torque[\1]", filedata)
filedata = re.sub("V2s\[(\d||\d\d), 0\]", r"V2s[\1]", filedata)
filedata = re.sub("F2s\[(\d||\d\d), 0\]", r"F2s[\1]", filedata)

with open(ME_file, 'w') as file:
    file.write(filedata)

______________________________________
_______________________

In [12]:
## try to assemble one of the matrices

In [13]:
matG1s1s = Matrix.zeros(3,3)
for i in range(3):
    for j in range(3):
        matG1s1s[i,j] = simplify(G1s1s(rij, i, j))

In [14]:
matG1s1s ##works and as far as I can see, matches the Mathematica result

Matrix([
[(-2*b**2*(2*xij**2 - yij**2 - zij**2) + 3*(xij**2 + yij**2 + zij**2)*(2*xij**2 + yij**2 + zij**2))/(24*pi*eta*(xij**2 + yij**2 + zij**2)**(5/2)),                                                         xij*yij*(-2*b**2 + xij**2 + yij**2 + zij**2)/(8*pi*eta*(xij**2 + yij**2 + zij**2)**(5/2)),                                                         xij*zij*(-2*b**2 + xij**2 + yij**2 + zij**2)/(8*pi*eta*(xij**2 + yij**2 + zij**2)**(5/2))],
[                                                       xij*yij*(-2*b**2 + xij**2 + yij**2 + zij**2)/(8*pi*eta*(xij**2 + yij**2 + zij**2)**(5/2)), (-2*b**2*(-xij**2 + 2*yij**2 - zij**2) + 3*(xij**2 + yij**2 + zij**2)*(xij**2 + 2*yij**2 + zij**2))/(24*pi*eta*(xij**2 + yij**2 + zij**2)**(5/2)),                                                         yij*zij*(-2*b**2 + xij**2 + yij**2 + zij**2)/(8*pi*eta*(xij**2 + yij**2 + zij**2)**(5/2))],
[                                                       xij*zij*(-2*b**2 + xij**2 + yij**2 + zij**2)/(8*pi*

In [15]:
## when printing to python code, might have to be careful with greek characters that sympy is printing so nicely
pycode(matG1s1s)

'ImmutableDenseMatrix([[(1/24)*(-2*b**2*(2*xij**2 - yij**2 - zij**2) + 3*(xij**2 + yij**2 + zij**2)*(2*xij**2 + yij**2 + zij**2))/(math.pi*eta*(xij**2 + yij**2 + zij**2)**(5/2)), (1/8)*xij*yij*(-2*b**2 + xij**2 + yij**2 + zij**2)/(math.pi*eta*(xij**2 + yij**2 + zij**2)**(5/2)), (1/8)*xij*zij*(-2*b**2 + xij**2 + yij**2 + zij**2)/(math.pi*eta*(xij**2 + yij**2 + zij**2)**(5/2))], [(1/8)*xij*yij*(-2*b**2 + xij**2 + yij**2 + zij**2)/(math.pi*eta*(xij**2 + yij**2 + zij**2)**(5/2)), (1/24)*(-2*b**2*(-xij**2 + 2*yij**2 - zij**2) + 3*(xij**2 + yij**2 + zij**2)*(xij**2 + 2*yij**2 + zij**2))/(math.pi*eta*(xij**2 + yij**2 + zij**2)**(5/2)), (1/8)*yij*zij*(-2*b**2 + xij**2 + yij**2 + zij**2)/(math.pi*eta*(xij**2 + yij**2 + zij**2)**(5/2))], [(1/8)*xij*zij*(-2*b**2 + xij**2 + yij**2 + zij**2)/(math.pi*eta*(xij**2 + yij**2 + zij**2)**(5/2)), (1/8)*yij*zij*(-2*b**2 + xij**2 + yij**2 + zij**2)/(math.pi*eta*(xij**2 + yij**2 + zij**2)**(5/2)), (1/24)*(-2*b**2*(-xij**2 - yij**2 + 2*zij**2) + 3*(xij**2 + y

In [16]:
from sympy.printing.numpy import NumPyPrinter

In [17]:
NumPyPrinter().doprint(matG1s1s)

'numpy.array([[(1/24)*(-2*b**2*(2*xij**2 - yij**2 - zij**2) + 3*(xij**2 + yij**2 + zij**2)*(2*xij**2 + yij**2 + zij**2))/(numpy.pi*eta*(xij**2 + yij**2 + zij**2)**(5/2)), (1/8)*xij*yij*(-2*b**2 + xij**2 + yij**2 + zij**2)/(numpy.pi*eta*(xij**2 + yij**2 + zij**2)**(5/2)), (1/8)*xij*zij*(-2*b**2 + xij**2 + yij**2 + zij**2)/(numpy.pi*eta*(xij**2 + yij**2 + zij**2)**(5/2))], [(1/8)*xij*yij*(-2*b**2 + xij**2 + yij**2 + zij**2)/(numpy.pi*eta*(xij**2 + yij**2 + zij**2)**(5/2)), (1/24)*(-2*b**2*(-xij**2 + 2*yij**2 - zij**2) + 3*(xij**2 + yij**2 + zij**2)*(xij**2 + 2*yij**2 + zij**2))/(numpy.pi*eta*(xij**2 + yij**2 + zij**2)**(5/2)), (1/8)*yij*zij*(-2*b**2 + xij**2 + yij**2 + zij**2)/(numpy.pi*eta*(xij**2 + yij**2 + zij**2)**(5/2))], [(1/8)*xij*zij*(-2*b**2 + xij**2 + yij**2 + zij**2)/(numpy.pi*eta*(xij**2 + yij**2 + zij**2)**(5/2)), (1/8)*yij*zij*(-2*b**2 + xij**2 + yij**2 + zij**2)/(numpy.pi*eta*(xij**2 + yij**2 + zij**2)**(5/2)), (1/24)*(-2*b**2*(-xij**2 - yij**2 + 2*zij**2) + 3*(xij**2 + yi